# Commit Message Exploration with SpaCy

### Load Data

In [1]:
import pandas as pd

data = pd.read_csv("results.csv")

,hash,diff,message,author_email,author_name,committer_email,committer_name,project,split
0,1de640cc59b4b3030447d567b3c99c50777bd760,a/setup.py b/setup.py\nindex <HASH>..<HASH> 1...,setup: Detect if wheel and twine installed,gcushen@users.noreply.github.com,George Cushen,gcushen@users.noreply.github.com,George Cushen,gcushen_mezzanine-api,train
1,c1cce6fe5e49df5546c30a662fd141d41f4fc389,a/Builder.php b/Builder.php\nindex <HASH>..<H...,[Builder] Adding root page in any case,g.passault@gmail.com,Gregwar,g.passault@gmail.com,Gregwar,Gregwar_Slidey,train
2,2f7d97d15ea41f4112e74429617c5daad740d7cc,a/web.go b/web.go\nindex <HASH>..<HASH> 10064...,Added web.Urlencode method,hoisie@gmail.com,Michael Hoisie,hoisie@gmail.com,Michael Hoisie,hoisie_web,train


## Exploration with spaCy

In [17]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [18]:
from spacy import displacy

print("Structure of first commit message as an example")
displacy.render(nlp(data["message"][0]))

Structure of first commit message as an example


Take a subset because unable to process all data locally.

In [19]:
subset_size = 100000

messages = data["message"][:subset_size].tolist()

In [23]:
first_tokens = []
tokens = []
token_tags = []
cats = []
ents = []
sentiments = []
polarities = []
subjectivities = []
imperative_count = []

def token_filter(token):
    return not token.is_stop and token.is_alpha

docs = nlp.pipe(messages)

from spacytextblob.spacytextblob import SpacyTextBlob

# the following installations are required
# python -m textblob.download_corpora
# python -m spacy download en_core_web_sm

nlp.add_pipe('spacytextblob')

for doc in docs:
    first_tokens.append(doc[0].lemma_)
    tokens.extend([token.lemma_ if token_filter(token) else None for token in doc])
    token_tags.extend([token.tag_ for token in doc])
    cats.extend(doc.cats if doc.cats != "" else "")
    ents.extend([ent.lemma_ for ent in doc.ents])
    sentiments.append(doc.sentiment)
    polarities.append(doc._.blob.polarity)
    subjectivities.append(doc._.blob.subjectivity)
    imperative_count.extend([token.morph for token in doc])

### First Tokens of a Commit Message

In [24]:
print("20 most common first (lemmatized) tokens in first {subset_size} messages:".format(subset_size = subset_size))

first_tokens_count = Counter(first_tokens)
first_tokens_count.most_common(20)

20 most common first (lemmatized) tokens in first 100000 messages:


[('fix', 12120),
 ('add', 11359),
 ('remove', 4198),
 ('[', 3935),
 ('update', 3218),
 ('use', 2106),
 ('do', 2047),
 ('make', 1876),
 ('Fix', 1757),
 ('change', 1635),
 ('allow', 1022),
 ('test', 980),
 ('bump', 657),
 ('move', 600),
 ('#', 539),
 ('check', 533),
 ('ensure', 530),
 ('set', 522),
 ('only', 492),
 ('bugfix', 472)]

### Tokens of a Commit Message

In [25]:
print("20 most common (lemmatized) alphabetical tokens in first {subset_size} messages excluding stopwords:".format(subset_size = subset_size))

tokens_count = Counter(tokens)
tokens_count.most_common(21)[1:]

20 most common (lemmatized) alphabetical tokens in first 100000 messages excluding stopwords:


[('fix', 20749),
 ('add', 17329),
 ('test', 11266),
 ('remove', 7423),
 ('use', 6156),
 ('update', 5640),
 ('change', 5470),
 ('method', 5208),
 ('error', 5186),
 ('url', 3951),
 ('file', 3932),
 ('check', 3768),
 ('version', 3606),
 ('issue', 3549),
 ('set', 3423),
 ('support', 3040),
 ('bug', 3003),
 ('return', 2833),
 ('Fix', 2655),
 ('code', 2644)]

### Token Tags

In [26]:
print("Most common token tags in first {subset_size} messages:".format(subset_size = subset_size))

token_tags_count = Counter(token_tags)
token_tags_count.most_common(10)

Most common token tags in first 100000 messages:


[('NN', 294339),
 ('IN', 137675),
 ('VB', 94535),
 ('NNP', 90266),
 ('DT', 75444),
 ('JJ', 72476),
 ('NNS', 70804),
 ('XX', 62388),
 ('RB', 51032),
 ('_SP', 46894)]

Check meaning of token tags with:

In [27]:
spacy.explain("JJ")

'adjective (English), other noun-modifier (Chinese)'

### Categories

In [28]:
print("Spacy finds the following categories: " + str(cats))
print("(Not expected to find any categories)")

Spacy finds the following categories: []
(Not expected to find any categories)


### Entities

In [29]:
ents_count = Counter(ents)
print("20 most common (lemmatized) entities in first {subset_size} messages:".format(subset_size = subset_size))
ents_count.most_common(20)

20 most common (lemmatized) entities in first 100000 messages:


[('first', 867),
 ('fix', 716),
 ('one', 666),
 ('fix #', 580),
 ('1', 477),
 ('2', 473),
 ('#', 447),
 ('0', 433),
 ('3', 429),
 ('api', 395),
 ('two', 370),
 ('Python', 321),
 ('API', 294),
 ('doc', 293),
 ('CI', 251),
 ('PHP', 236),
 ('improve', 234),
 ('second', 223),
 ('json', 200),
 ('zero', 193)]

### Sentiments

In [30]:
print_statistics(sentiments)

Mean: 0.0
Standard deviation: 0.0
Minimum: 0.0
25 % quartile: 0.0
Median: 0.0
75 % quartile: 0.0
Maximum: 0.0



Spacy Sentiment is always zero.

### Polarities

In [31]:
print_statistics(polarities)

Mean: 0.012914568989495398
Standard deviation: 0.16576854685975387
Minimum: -1.0
25 % quartile: 0.0
Median: 0.0
75 % quartile: 0.0
Maximum: 1.0



There are only minor polarities found in the commit messages.

### Subjectivities

In [32]:
print_statistics(subjectivities)

Mean: 0.17876842287919062
Standard deviation: 0.27419620402845424
Minimum: 0.0
25 % quartile: 0.0
Median: 0.0
75 % quartile: 0.3333333333333333
Maximum: 1.0



Most commit messages are objective (subjectivity score of 0.0).

### Imperative Counts

In [33]:
imperative_count

[Number=Sing,
 ,
 VerbForm=Inf,
 ,
 Number=Sing,
 ConjType=Cmp,
 VerbForm=Inf,
 Aspect=Perf|Tense=Past|VerbForm=Part,
 ,
 Number=Sing,
 ,
 Aspect=Prog|Tense=Pres|VerbForm=Part,
 Number=Sing,
 Number=Sing,
 ,
 ,
 Number=Sing,
 Aspect=Perf|Tense=Past|VerbForm=Part,
 Number=Sing,
 PunctType=Peri,
 Number=Sing,
 Number=Sing,
 VerbForm=Inf,
 ,
 PunctType=Dash,
 Aspect=Perf|Tense=Past|VerbForm=Part,
 Number=Plur,
 ,
 Case=Nom|Person=2|PronType=Prs,
 Mood=Ind|Tense=Pres|VerbForm=Fin,
 Polarity=Neg,
 VerbForm=Inf,
 Number=Sing,
 Number=Sing,
 ConjType=Cmp,
 Degree=Pos,
 Number=Plur,
 PunctType=Peri,
 Number=Sing,
 ,
 Number=Sing,
 Number=Plur,
 VerbForm=Fin,
 VerbForm=Inf,
 Definite=Def|PronType=Art,
 Number=Sing,
 ,
 ,
 Degree=Pos,
 Degree=Pos,
 Number=Plur,
 Number=Sing|Person=3|Tense=Pres|VerbForm=Fin,
 ,
 VerbForm=Inf,
 Definite=Ind|PronType=Art,
 Degree=Cmp,
 Number=Sing,
 PunctType=Peri,
 VerbForm=Inf,
 ,
 Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin,
 Aspect=Perf|Tense=Past|Ver

Spacy seems not to be able to detect imperatives.